# **PROJECT FAKE NEWS DETECTION WITH MULTIPLE MODEL AND LANGUAGE AWARENESS**

In [1]:
!python --version


Python 3.11.13


In [2]:
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2
!pip install transformers==4.36.2 datasets==2.16.1 \
sentencepiece==0.1.99 tokenizers==0.15.0
!pip install langdetect==1.0.9 indic-nlp-library==0.92
!pip install pandas==2.1.4 numpy==1.26.4 openpyxl==3.1.2 \
scikit-learn==1.3.2 tqdm==4.66.1 evaluate==0.4.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

^C


In [1]:
import torch, transformers, sentencepiece, tokenizers
print("Python OK")
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Transformers:", transformers.__version__)
print("Tokenizers:", tokenizers.__version__)


Python OK
Torch: 2.0.1+cu117
CUDA available: True
Transformers: 4.36.2
Tokenizers: 0.15.0


In [2]:
import pandas as pd

df = pd.read_excel("bharatfakenewskosh.xlsx")
print(df.shape)
df.head()


(26232, 19)


,id,Author_Name,Fact_Check_Source,Source_Type,Statement,Eng_Trans_Statement,News Body,Eng_Trans_News_Body,Media_Link,Publish_Date,Fact_Check_Link,News_Category,Language,Region,Platform,Text,Video,Image,Label
0,BFNK_1,Shinjinee Majumder,Alt News,IFCN,फ़ैक्ट-चेक: तेलंगाना में एक रिपोर्टर ने गृह मंत...,Fact-check: A reporter in Telangana stopped sp...,सोशल मीडिया पर एक वीडियो वायरल है जिसमें एक पत...,A video is viral on social media in which a jo...,https://i0.wp.com/www.altnews.in/Hindi/wp-cont...,9th July 2022,https://www.altnews.in/Hindi/fact-check-was-am...,Politics,Hindi,Telangana,Twitter,no,yes,no,False
1,BFNK_2,Kalim Ahmed,Alt News,IFCN,PM मोदी को UAE का सर्वोच्च नागरिक सम्मान मिलने...,Share by stating the old video of PM Modi's hi...,प्रधानमंत्री नरेंद्र मोदी को सोने की चेन से सम...,A video of Prime Minister Narendra Modi being ...,https://i0.wp.com/www.altnews.in/Hindi/wp-cont...,9th July 2022,https://www.altnews.in/Hindi/old-video-of-pm-m...,Politics,Hindi,National,Twitter,no,yes,no,False
2,BFNK_3,Abhishek Kumar,Alt News,IFCN,वायरल तस्वीर में सुप्रीम कोर्ट के जज सूर्यकांत...,Supreme Court judges Suryakant and JB Pardiwal...,बीते दिनों नूपुर शर्मा ने टीवी डिबेट में पैगम्...,"Recently, Nupur Sharma made an objectionable c...",https://i0.wp.com/www.altnews.in/Hindi/wp-cont...,7th July 2022,https://www.altnews.in/Hindi/false-claim-with-...,Politics,Hindi,National,Twitter,no,no,yes,False
3,BFNK_4,Abhishek Kumar,Alt News,IFCN,मीडिया ने दी ग़लत ख़बर: कटनी में मुस्लिम सरपंच क...,Media gave wrong news: After the victory of Mu...,एक वीडियो सोशल मीडिया पर वायरल है. इसे शेयर कर...,A video is viral on social media. While sharin...,https://i0.wp.com/www.altnews.in/Hindi/wp-cont...,5th July 2022,https://www.altnews.in/Hindi/media-misreport-p...,Politics,Hindi,Madhya Pradesh,Twitter,no,no,yes,False
4,BFNK_5,Kinjal,Alt News,IFCN,महिला ने राहुल गांधी को कश्मीर मुद्दे पर मोदी ...,The woman lashed out at Rahul Gandhi to oppose...,सोशल मीडिया पर राहुल गांधी का एक वीडियो वायरल ...,A video of Rahul Gandhi has gone viral on soci...,https://i0.wp.com/www.altnews.in/Hindi/wp-cont...,4th July 2022,https://www.altnews.in/Hindi/2019-video-of-wom...,Politics,Hindi,Kashmir,Twitter,no,yes,no,True


In [3]:
df.columns

Index(['id', 'Author_Name', 'Fact_Check_Source', 'Source_Type', 'Statement',
       'Eng_Trans_Statement', 'News Body', 'Eng_Trans_News_Body', 'Media_Link',
       'Publish_Date', 'Fact_Check_Link', 'News_Category', 'Language',
       'Region', 'Platform', 'Text', 'Video', 'Image', 'Label'],
      dtype='object')

In [4]:
df['Language'].value_counts()

,count
Language,
English,10010
Tamil,4757
Hindi,4192
Malayalam,2723
Gujarati,2205
Odia,1268
Bangla,584
Assamese,409
Telugu,84


In [5]:
df['label'] = (df['Label'].astype(str).str.upper() == 'FALSE').astype(int)

In [6]:
df['label'].value_counts()

,count
label,
0,15913
1,10319


In [7]:
# English data
df_en = df[df['Language'] == 'English']

# Indic languages (everything except English)
df_indic = df[df['Language'] != 'English']

print("English samples:", len(df_en))
print("Indic samples:", len(df_indic))

English samples: 10010
Indic samples: 16222


In [8]:
from sklearn.model_selection import train_test_split

# English
X_en_train, X_en_test, y_en_train, y_en_test = train_test_split(
    df_en['Eng_Trans_News_Body'],
    df_en['label'],
    test_size=0.2,
    stratify=df_en['label'],
    random_state=42
)

# Indic
X_in_train, X_in_test, y_in_train, y_in_test = train_test_split(
    df_indic['News Body'],
    df_indic['label'],
    test_size=0.2,
    stratify=df_indic['label'],
    random_state=42
)

In [9]:
import torch
from torch.utils.data import Dataset

class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts.tolist()
        self.labels = labels.tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx])
        }


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

en_model_name = "distilbert-base-uncased"
en_tokenizer = AutoTokenizer.from_pretrained(en_model_name)
en_model = AutoModelForSequenceClassification.from_pretrained(en_model_name, num_labels=2)

train_en_ds = NewsDataset(X_en_train, y_en_train, en_tokenizer)
test_en_ds = NewsDataset(X_en_test, y_en_test, en_tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
indic_model_name = "ai4bharat/indic-bert"
indic_tokenizer = AutoTokenizer.from_pretrained(indic_model_name)
indic_model = AutoModelForSequenceClassification.from_pretrained(indic_model_name, num_labels=2)

train_in_ds = NewsDataset(X_in_train, y_in_train, indic_tokenizer)
test_in_ds = NewsDataset(X_in_test, y_in_test, indic_tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
indic_model=indic_model.to(device)

In [20]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary'
    )
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [21]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_in_ds, batch_size=8, shuffle=True)
test_loader = DataLoader(test_in_ds, batch_size=8)

In [23]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0, 1]),
    y=y_in_train
)

class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print(class_weights)


tensor([0.8221, 1.2763], device='cuda:0')


In [26]:
from torch.optim import AdamW

optimizer = AdamW(indic_model.parameters(), lr=1e-5)

epochs = 5

for epoch in range(epochs):
    indic_model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = indic_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(outputs.logits, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f}")



ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

indic_model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = indic_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        predictions = torch.argmax(outputs.logits, dim=1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print("Accuracy:", accuracy_score(true_labels, preds))
print(classification_report(true_labels, preds))
print("Confusion Matrix:\n", confusion_matrix(true_labels, preds))


In [ ]:
indic_model.save_pretrained("fake_news_indicbert")
indic_tokenizer.save_pretrained("fake_news_indicbert")


In [ ]:
!zip -r fake_news_indicbert.zip fake_news_indicbert


In [ ]:
from google.colab import files
files.download("fake_news_indicbert.zip")


**DIstilbert for English **

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
en_model.to(device)


In [ ]:
from torch.utils.data import DataLoader

train_en_loader = DataLoader(train_en_ds, batch_size=8, shuffle=True)
test_en_loader = DataLoader(test_en_ds, batch_size=8)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights_en = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0, 1]),
    y=y_en_train
)

class_weights_en = torch.tensor(class_weights_en, dtype=torch.float).to(device)
print(class_weights_en)


In [ ]:
from torch.optim import AdamW

optimizer = AdamW(en_model.parameters(), lr=1e-5)
loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_en)

epochs = 5

for epoch in range(epochs):
    en_model.train()
    total_loss = 0

    for batch in train_en_loader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = en_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = loss_fct(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"English Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_en_loader):.4f}")


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

en_model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in test_en_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = en_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        predictions = torch.argmax(outputs.logits, dim=1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print("Accuracy:", accuracy_score(true_labels, preds))
print(classification_report(true_labels, preds))
print("Confusion Matrix:\n", confusion_matrix(true_labels, preds))


In [ ]:
en_model.save_pretrained("fake_news_distilbert")
en_tokenizer.save_pretrained("fake_news_distilbert")


In [ ]:
!zip -r fake_news_distilbert.zip fake_news_distilbert


In [ ]:
from google.colab import files
files.download("fake_news_distilbert.zip")


In [ ]:
def predict_news_final(text):
    try:
        lang = detect(text)
    except:
        lang = "unknown"

    if lang == "en":
        model = en_model
        tokenizer = en_tokenizer
    else:
        model = indic_model
        tokenizer = indic_tokenizer

    model.eval()

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)

    real_p = probs[0][0].item()
    fake_p = probs[0][1].item()

    if fake_p >= 0.7:
        label = "FAKE NEWS"
    elif fake_p >= 0.4:
        label = "SUSPICIOUS / NEEDS FACT CHECK"
    else:
        label = "LIKELY REAL NEWS"

    return {
        "language": lang,
        "real_probability": round(real_p, 3),
        "fake_probability": round(fake_p, 3),
        "final_decision": label
    }


In [ ]:
predict_news_final("The Reserve Bank of India announced a new policy to control inflation and stabilize the economy.")

In [ ]:
predict_news_final("Drinking warm water every hour can completely cure COVID-19, doctors confirm.")

In [ ]:
predict_news_final("Government announces free smartphones and laptops for every citizen starting next week.")

In [ ]:
predict_news_final("A viral video of PM shows disrespecting the National Flag")

In [ ]:
predict_news_final("A single spoon of this homemade remedy can cure diabetes permanently within 7 days, doctors shocked.")